# Setup

In [1]:
using Pkg

In [2]:
Pkg.status()

Status `~/.julia/environments/v1.5/Project.toml`
  [0c46a032] DifferentialEquations v6.16.0
  [31c24e10] Distributions v0.24.12
  [38e38edf] GLM v1.3.11
  [7073ff75] IJulia v1.23.1
  [a98d9a8b] Interpolations v0.13.1
  [c7f686f2] MCMCChains v4.6.0
  [6f286f6a] MultivariateStats v0.7.0
  [65888b18] ParameterizedFunctions v5.9.0
  [91a5bcdd] Plots v1.10.2
  [f3b207a7] StatsPlots v0.14.17
  [9f7883ad] Tracker v0.2.15
  [fce5fe82] Turing v0.15.8


In [3]:
Pkg.status("DiffEqBase")

No Matches in `~/.julia/environments/v1.5/Project.toml`


In [4]:
Pkg.add(Pkg.PackageSpec(;name="DiffEqBase", version="6.57.1"))

   Updating registry at `~/.julia/registries/General`
tar: Failed to set default locale
tar: Failed to set default locale
######################################################################### 100.0%
tar: Failed to set default locale
  Resolving package versions...
tar: Failed to set default locale
  Installed DiffEqBase ─ v6.57.1
Updating `~/.julia/environments/v1.5/Project.toml`
  [2b5f629d] + DiffEqBase v6.57.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [2b5f629d] ↑ DiffEqBase v6.53.6 ⇒ v6.57.1


In [5]:
using DifferentialEquations, Plots, Turing, Interpolations, StatsPlots

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1278


# Three node network

In [6]:
phi(a::Number) = (exp(2*a)-1)/(exp(2*a)+1)

phi (generic function with 1 method)

In [ ]:
function additive_noise!(du,u,p,t)
#     s,g,W,I,ϕ = p
    s,g,W,ϕ = p
    for i = 1:length(u)
        du[i] = ϕ #This value multiplies a random number drawn from N(0, dt) i.e. noise then scales with sampling rate and effective sd of noise will be e.g. 0.1*dt
    end
end

function network_model!(du,u,p,t)
#     s,g,W,I,ϕ = p
    s,g,W,ϕ = p
    for i=1:length(u)
#         du[i] = -u[i] + s*phi(u[i]) + g*(sum(W[i,:].*u)) + I(t)
        du[i] = -u[i] + s*phi(u[i]) + g*(sum(W[i,:].*u))
    end
end

In [ ]:
num_nodes = 3
u0 = repeat([0.0], num_nodes)
Tmax = 100.0
tspan = (0,Tmax)

# s = 0.3
# g = 0.7
# W = [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0]

ts = 1:1:Tmax
task = repeat([0.0], 100)
# task[4] = 1 #without impulse it'd be "resting state"
# I = LinearInterpolation(ts, task, extrapolation_bc = 0)
dt = 0.5
# ϕ = 0.1

# p = [s,g,W,I,ϕ]
# p = [0.3, 0.7, [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0], LinearInterpolation(ts, task, extrapolation_bc = 0), 0.1]
p = [0.3, 0.7, [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0], 0.1]

prob_sde = SDEProblem(network_model!,additive_noise!,u0,tspan,p)


Note: If you don't turn off adaptive step size then the number of solved points will not be Tmax/dt. It will be determined as detailed [here](https://diffeq.sciml.ai/stable/basics/common_solver_opts/#Basic-Stepsize-Control)

In [ ]:
# sol = solve(prob_sde, dt=dt, adaptive=false)

In [ ]:
# plot(sol)

In [ ]:
ensembleprob = EnsembleProblem(prob_sde)
# data = solve(ensembleprob, SOSRI(), trajectories=100, dt=dt, adaptive=false, saveat=dt)
data = solve(ensembleprob, SOSRI(), trajectories=100)
plot(EnsembleSummary(data))

In [ ]:
Turing.setadbackend(:forwarddiff)
@model function fit_nmm(data, prob, ::Type{T} = Float64) where {T}
    σ ~ InverseGamma(2,3)
    ϕ ~ truncated(Normal(0.0,1.1),-3.0,3.0)
#     ϕ = 0.1
    s = 0.3
    g = 0.7
    W = [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0]
#     ts = 1:1:Tmax
#     task = repeat([0.0], 100)
#     I = LinearInterpolation(ts, task, extrapolation_bc = 0)
#     p = [s,g,W,I,ϕ]
    p = [s,g,W,ϕ]
    prob = remake(prob, p=p, saveat=dt)
    predicted = solve(prob, SOSRI(), dt=dt, adaptive=false)

#     if predicted.retcode != :Success
#         Turing.acclogp!(_varinfo, -Inf)
#     end
    for j in 1:length(data) #number of ensemble iterations
        for i = 1:length(predicted) #number of time points
            data[j][i] ~ MvNormal(predicted[i],σ) #generate 3 points at a time for each node
        end
    end
end;

In [ ]:
model = fit_nmm(data, prob_sde)

In [ ]:
@code_warntype model.f(
    Random.GLOBAL_RNG,
    model,
    Turing.VarInfo(model),
    Turing.SampleFromPrior(),
    Turing.DefaultContext(),
    model.args...,
)

In [ ]:
chain = sample(model, NUTS(0.25), 100)

In [ ]:
plot(chain)

In [ ]:
typeof(u0)